### Importing Packages

In [43]:
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableBranch
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

True

### Model Setup

In [29]:
model1 = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.7,
    max_tokens=1024,
)

model2 = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
    max_tokens=1024
)

### Simple Chain

In [30]:
parser = StrOutputParser()

prompt = PromptTemplate(
    template='Generate 5 interesting and unique facts about {topic}',
    input_variables=['topic']
)

chain = prompt | model1 | parser

In [31]:
result = chain.invoke({'topic':'Computer Science'})
print(result)

Here are 5 interesting and unique facts about Computer Science:

1. **The First Computer Bug Was an Actual Insect**: In 1947, a team of engineers working on the Harvard Mark II computer discovered a moth stuck in one of the relays. They taped the moth to the computer log and wrote "First actual case of bug being found" next to it. From then on, any technical issue with the computer was referred to as a "bug." This story highlights the often-overlooked history and anecdotes that underlie the development of computer science.

2. **The Concept of Artificial Intelligence Has Been Around Since Ancient Greece**: While computer science as we know it today is a relatively recent development, the idea of creating machines that can think and behave like humans dates back to ancient times. The Greek myth of Talos, a bronze giant who was said to be able to think and act, is an early example of this idea. The modern field of artificial intelligence (AI) draws on this long history of exploring the p

In [21]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


### Sequential Chain

In [32]:
parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

chain = prompt1 | model1 | parser | prompt2 | model1 | parser

In [25]:
result = chain.invoke({'topic': 'Unemployment due to AI in Software Engineering'})
print(result)

Here is a 5-pointer summary of the report on "Unemployment due to AI in Software Engineering: A Detailed Report":

1. **AI is increasingly being used in software engineering**: AI-powered tools are being used to automate tasks such as code completion, code review, testing, and DevOps, posing a risk to employment in the industry.
2. **Certain tasks and roles are more susceptible to automation**: Junior developers, QA engineers, code reviewers, and technical writers are among the roles that are most likely to be displaced by AI-powered tools.
3. **Consequences of unemployment due to AI could be significant**: Job displacement, skills obsolescence, and economic disruption are potential consequences of widespread unemployment in the software engineering industry due to AI.
4. **Mitigating measures can be taken**: Upskilling and reskilling, education and training, industry support, and government initiatives can help mitigate the negative consequences of unemployment due to AI.
5. **Collabo

In [26]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

### Parallel Chain

In [ ]:
parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

In [35]:
parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

In [37]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})
print(result)

Here is the merged document:

**Support Vector Machines (SVMs) Notes and Quiz**

**What is SVM?**
- Supervised learning method
- Used for: classification, regression, outliers detection

**Advantages:**
1. Effective in high dimensional spaces
2. Memory efficient (uses support vectors)
3. Versatile (different kernel functions)

**Disadvantages:**
1. Risk of over-fitting if many features, few samples
2. Doesn't directly provide probability estimates

**Using SVM in scikit-learn:**
- Supports dense and sparse input vectors
- Optimal performance: C-ordered numpy.ndarray or scipy.sparse.csr_matrix with dtype=float64

**Quiz: Support Vector Machines (SVMs)**

1. **Q: What are Support Vector Machines (SVMs)?**
 **A:** SVMs are supervised learning methods used for classification, regression, and outlier detection.

2. **Q: What is a key advantage of SVMs regarding high dimensionality?**
 **A:** SVMs are effective in high-dimensional spaces, even when the number of dimensions exceeds the number

In [36]:
chain.get_graph().print_ascii()

          +---------------------------+                
          | Parallel<notes,quiz>Input |                
          +---------------------------+                
                ***             ***                    
              **                   **                  
            **                       **                
+----------------+              +----------------+     
| PromptTemplate |              | PromptTemplate |     
+----------------+              +----------------+     
          *                             *              
          *                             *              
          *                             *              
    +----------+            +------------------------+ 
    | ChatGroq |            | ChatGoogleGenerativeAI | 
    +----------+            +------------------------+ 
          *                             *              
          *                             *              
          *                             *       

### Conditional Chain

In [39]:
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)
parser = StrOutputParser()

In [40]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [44]:
classifier_chain = prompt1 | model1 | parser2

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model1 | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model1 | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

In [45]:
chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

Here is a potential response:

"Thank you so much! I'm glad to hear that you're happy. Your satisfaction is my top priority, and I'm thrilled that I could meet your expectations. I appreciate your feedback and look forward to serving you again in the future!"

This response acknowledges the customer's positive sentiment, expresses gratitude, and shows appreciation for their feedback. 

If you want a shorter response, here are a few alternatives:

* "Thank you! I'm glad you're happy."
* "Thanks for your kind words! I'm glad I could meet your expectations."
* "Appreciate your feedback! Glad you're satisfied." 

Let me know if you need any adjustments.


In [46]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
